 # 1. DEFINE I/O DATA FRAMES

In [32]:
import numpy as np 
import pandas as pd 

MODEL_NAME='multi_bibasic_lstm'
HPARAMS_STR='lr5.00e-04,hs128x64'
MODEL_V= '12'
INPUT_V='00'
INPUT_DIR= '../datasets/csvs/'
OUTPUT_DIR='../outputs/'

INPUT_PATH = '{:}{:}'.format(INPUT_DIR, 'zhou_valid.csv')
OUTPUT_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'Yhat_valid.csv')
TARGET_PATH= '{:}{:}/{:}/{:}/{:}'.format(OUTPUT_DIR,MODEL_NAME,HPARAMS_STR,MODEL_V, 'evaluation_valid.csv')



print(INPUT_PATH)
print(OUTPUT_PATH)

../datasets/csvs/zhou_valid.csv
../outputs/multi_bibasic_lstm/lr5.00e-04,hs128x64/12/Yhat_valid.csv


In [21]:
df_input= pd.read_csv(INPUT_PATH, index_col=0)
df_output=pd.read_csv(OUTPUT_PATH, index_col=0)
print(df_input.shape, df_output.shape)

(12298, 10) (12298, 1)


In [22]:
df= df_input.join(df_output, how='left')
print(df.shape)

(12298, 11)


In [23]:
df.to_csv(TARGET_PATH)

In [24]:
df.head()

,ID,S,P,P_S,FORM,LEMMA,PRED,M_R,ARG,ARG_Y,Yhat
IDX,,,,,,,,,,,
123846,1,2879,1,5100,Entre,entre,estar,0,(A2*,A2,A2
123847,2,2879,1,5100,os,o,estar,0,*,A2,A2
123848,3,2879,1,5100,convidados,convidado,estar,0,*),A2,A1
123849,4,2879,1,5100,estão,estar,estar,1,(V*),V,V
123850,5,2879,1,5100,o,o,estar,1,(A1*,A1,A2


In [30]:
df2= df[['ARG_Y','Yhat']]
table= pd.pivot_table(df2.iloc[:30,:], index='ARG_Y', columns='Yhat', aggfunc=len, fill_value=0)
table

Yhat,*,A0,A1,A2,V
ARG_Y,,,,,
*,1,0,0,0,0
A0,0,1,0,0,0
A1,0,0,22,2,0
A2,0,0,1,2,0
V,0,0,0,0,1


 # 2. CONFUSION MATRIX

In [58]:
df1= df[['ARG_Y','Yhat']]
table= pd.pivot_table(df1, index='ARG_Y', columns='Yhat', aggfunc=len, fill_value=0)
table

Yhat,*,A0,A1,A2,A3,A4,AM-ADV,AM-CAU,AM-DIR,AM-DIS,AM-EXT,AM-LOC,AM-MED,AM-MNR,AM-NEG,AM-PNC,AM-PRD,AM-REC,AM-TMP,V
ARG_Y,,,,,,,,,,,,,,,,,,,,
*,3645,154,204,97,4,11,31,1,0,2,0,19,0,11,4,1,20,0,17,22
A0,356,454,75,32,1,0,9,10,0,4,2,7,0,12,1,0,4,0,10,0
A1,711,103,1735,273,14,4,49,17,0,4,11,52,0,41,13,20,21,3,42,16
A2,369,51,346,508,0,6,2,3,4,0,0,5,0,6,0,4,9,1,12,23
A3,6,2,13,17,12,6,0,0,0,0,0,3,0,12,0,0,2,0,10,0
A4,2,0,4,9,0,20,0,0,0,0,0,7,0,0,0,2,0,0,1,0
AM-ADV,34,8,42,4,1,0,12,1,0,6,0,3,0,0,0,0,0,0,0,0
AM-CAU,71,13,28,7,0,0,0,14,0,0,0,1,4,3,0,0,5,0,6,1
AM-DIR,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
print(table.columns + table.index)

ValueError: operands could not be broadcast together with shapes (20,) (18,) 

 ## 2.1 RELATIVE FREQUENCIES ARG_Y
 ### True frequency

In [54]:
df2.groupby('ARG_Y').count() #np.sum(table.as_matrix(),axis=1)

,Yhat
ARG_Y,
*,4243
A0,977
A1,3129
A2,1349
A3,83
A4,45
AM-ADV,111
AM-CAU,153
AM-DIR,3


 ## 2.2 RELATIVE FREQUENCIES Yhat
 ### Forecasted frequency

In [57]:
df2.groupby('Yhat').count() #np.sum(table.as_matrix(),axis=0)

,ARG_Y
Yhat,
*,5803
A0,894
A1,2736
A2,1147
A3,45
A4,53
AM-ADV,123
AM-CAU,69
AM-DIR,11


In [50]:
A= table.as_matrix(); #converts to numpy
relevant_classes=set(table.index) # 18
retrieved_classes=set(table.columns)# 20


# diagA= np.diag(A)    # true positives
# rowsSumA= np.sum(A, axis=0) # relevant results 
# colsSumA= np.sum(A, axis=1) # retrieved results 
# df_precision= pd.DataFrame(data=diagA / colsSumA, index=table.index.tolist())
# df_precision.head(18)

print(set(table.columns))
print(set(table.index))


{'AM-CAU', 'A1', 'AM-LOC', 'AM-NEG', 'A3', 'AM-TMP', 'V', 'AM-REC', 'AM-MNR', 'AM-ADV', 'AM-PNC', '*', 'A4', 'AM-EXT', 'AM-MED', 'AM-DIR', 'AM-PRD', 'AM-DIS', 'A0', 'A2'}
{'AM-CAU', 'A1', 'AM-LOC', 'AM-NEG', 'A3', 'AM-TMP', 'V', 'AM-MNR', 'AM-ADV', 'AM-PNC', '*', 'A4', 'AM-EXT', 'AM-PRD', 'AM-DIR', 'AM-DIS', 'A0', 'A2'}


 ## 3 Precision
 ### How many selected items are relevant?

 ## 2.2 Recall

In [45]:
# df_recall= pd.DataFrame(data=diagA /rowsSumA, index=table.index.tolist())
# df_recall.head(18)

print(len(rowsSumA))

20


## 2.3 F1 score